## DAX Summary

### syntax note


#### heading 4


**bold**


*italicized text*
1. First item
2. Second item
3. Third item


- first item
- second item
- third item


`percetage = calclate...`



### Table Relationship

1. avoid the use of cross direction filter
2. try to build one-to-many relationship rather than 1-to-1 or many-to-many relationships

DAX and SQL handle relationship in different ways.
Example: 2 Tables: customer and sales

in SQL to get the total sale amount for each customer


    SELECT
        Customer.CustomerName, 
        SUM (Sales.SaleAmount) As SumofAsles
    FROM
        Sales
    INNER JOIN Customers
        ON Sales.CustomerKey = Customer.CustomerKey
    Group By
        Coutomers.CustomerName


IN DAX:

    Evaluate 
    Summarizecolumns(
    Customer[CustomerName],
    "SumOfSales", SUM(Sales[SalesAmout])
    }

it will automatically apply **left out joins**

`Summarizacolumns` will do a group by to customers based customer name

- SQL: declarative language -- can trust the query optimizaer 
- DAX: functional language -- need to write good code to improve efficiency
- - DAX makes no difference between query and programming (??)
  - Programming Language: can use DAX to define calculated columns and calcualted tables
  - Query Language: can use DAX to retrieve data from a tabular model using reporting services.

- SQL can use sub query to implement more complex requirement, DAX can also do this

        SELECT
          CustomerName,
          SumOfSales
        From
          SELECT
              Customers.CustomerName,
              SUM(Sales.SealsAmount) AS SumOfSales
          From
            Sales
            Inner Join Cusomters
            On Sales.CustomerKey = Customers.CustomerKey
          Group by
          Customers.CustomerName
          ) AS SubQuery
          Where
          SubQuery.SumofSales > 100

      Evaluate
      Filter(
          SummarizeColumns(
              Customer[CustomerName],
              "SumOfSales", SUM(Sales[SalesAmount])),
          [SumofSales]> 100)


### Basic Concept

1. Column
    1. 'Table Name'[Column Name]    
       single quotation can be omited when (but not recommended, having the table name can help to distinguish measures and columns)
       1. referring a column inside the table where we define teh formula.
       2.  the tbale name not start with a number not include space or not a reserved word
  
2. Comments( not recomment to put comments at the end of DAX Expression
    1. single line comments using // `code`
    2. multiple line comments using /* `code` */
3. Data Type

4. Data Type: Integer,Decimal, Currency, DataTime, Boolean, String, Variant, Binary

    1. Currency: also known as Fixed Decimal Number, stores as a Fixed Decimal Number , represent **4 decimal point **. Summing or Subtracting Currency data types always ignore deciamls beyond the fourth decimal point, whereas multiplication and division produce a floating-points values, thue improve the precision of hte result. in general, use deicimal data type when need more accuracy.

   
        The default format of Currency data type includes the currncy symbol.


    2.  DateTime
    
        using a floating-point number internally. wherein the interger corresponds to the number of days since Dec 30 1989. and the decimal part identifies the fraction of the day. Hours, minutes, and seconds are converted to decimal fractions of a day. Thus, if you add a date by some integer, it will add by **DAYS** to this date.
    
           If only need the date part of a datetime, use **TRUNC** to get rid of hte decimal part.

           Power BI provide Date and Time data type, internally are simple version of DateTime. *i.e. the integer part and the decimal part*

           A potential bug in DAX: 1900 is considered as a leap year which is not correct. Offically the first date operated by DAX is 1 Mar 1990. thus it may cause problem when dealing with date before 1 Mar 1900.

    3. String: **NOT** Case Sensitive
  
    4. Variant: used for expressions that might return different data Types
  
           IF ([Measure] > 0, 1, "N/A")
  
       Variant Data type cannot be used as data type for a column in a regular tablbe, a DAX measure, and in general, a DAX Expression can be Variant
  
    5. Logical
  
       1. && And condition
       `[CountryRegion] = "USA" && [Quantity] > 0
       2. || OR Condition
       3. In
       4. NOT
       5. Example:
        `AND([CountryRegion] = "USA", [Quantity] >0) OR ([CountryRegion = "USA", [Quantity] >0)`
        6. Switch Function
            ```sql
            'Product'[SizeDesc] =
            SWITCH (
            'Product'[Size],
            "S", "Small",
            "M", "Medium",
            "L", "Large",
            "XL", "Extra Large",
            "Other"
            )
            ```


    
    6. Empty Values Handling
    
        1. Blank() usage
            ```sql
            BLANK () − 10 = −10
            18 + BLANK () = 18
            4 / BLANK () = Infinity
            0 / BLANK () = NaN
            BLANK () || BLANK () = FALSE
            BLANK () && BLANK () = FALSE
            ( BLANK () = BLANK () ) = TRUE
            ( BLANK () = TRUE ) = FALSE
            ( BLANK () = FALSE ) = TRUE
            ( BLANK () = 0 ) = TRUE
            ( BLANK () = "" ) = TRUE
            ISBLANK ( BLANK() ) = TRUE
            FALSE || BLANK () = FALSE
            FALSE && BLANK () = FALSE
            TRUE || BLANK () = TRUE
            TRUE && BLANK () = FALSE
            ```

    7. DAX Format

        1. 网址：www.daxformatter.com.
        2. column之前要加table名字，measure 之前不要加，为了帮助区分两者

    8. Relational Functions
        1. RELATED

            one-to-many relationship, RELATED can access the one-side from the many-side

        2. RELATEDTABLE
        
            uses on the one-side of the relationship and needs to access the many-side

            relatedtable is offen used in conjunction with iterators
            ```sql
            'Product Category'[CategorySales] = 
            SUMX(
                RELATEDTABLE(Sales),
                Sales[Quantity] * Sales[Net Price]
            )
            ```
    
       




#### Table Construction

 
   {} - Tables
   
   () - Rows.

   , - seperate columns.
   
`{ "Red", "Blue", "White" }`
   
`{( "Red" ), ( "Blue" ), ( "White" )}`
   
   The 2 Statements will generate the same result

   Example to construct a table:
   
        {( "A", 10, 1.5, DATE ( 2017, 1, 1 ), CURRENCY ( 199.99 ), TRUE ),
         ( "B", 20, 2.5, DATE ( 2017, 1, 2 ), CURRENCY ( 249.99 ), FALSE ),
         ( "C", 30, 3.5, DATE ( 2017, 1, 3 ), CURRENCY ( 299.99 ), FALSE )}


   **Usage:**

   Table construcor is commonly used with `IN` Operation to test if an element is in a set(table). But we cannot use `=` to compare between elements

   InValid:
   `('date[year],'date'[monthnumber]) = (2007, 12)`

   Valid:
   `('date[year],'date'[monthnumber]) in {(2007, 12)}`

   


#### Aggregator and Iterators

1. Aggregation Funciton 
    
    aggregation **column** values 

    SUM, AVERAGE, MIN, MAX...

    MIN & MAX can applicable for text values

2. Iterator:
    can aggretate an **expression** instead of **Column**
    ```sql
    AvgDelivery := AVERAGE ( Sales[DaysToDeliver] )

    AvgDelivery :=
    AVERAGEX (
    Sales,
    INT ( Sales[Delivery Date] - Sales[Order Date] )
    )
    ```
    在上面的例子中，Average是一个**aggregation**, 只可以用在已经存在的column上，AverageX是一个**iterator**，可以用在expression上面（比如对已有的column进行加减乘除）
    但是iterator不只是可以用在column上面，所以可以极大的减少创造expensive calculated column 的需求


1. 很多Iterator是有对应的aggregation function的

    like 

    MIN - MINX

    MAX - MAXX
    ...


2. 但是也有一些是完全没有对应的aggregation function的

    例如

    FILTER, ADDCOLUMNS, GENERATE

3. XXXA Functions
    AVERAGEA, COUNTA, MINA, MAXA

    same behavior like excel functions

    **will treat** 
    TRUE - 1
    FALSE - 0
    TEXT VALUES - 0

    建议不要用这些function

4. Count Funcitons

    1. **COUNT** operates on any data type, apart from Boolean.
    2. **COUNTA** operates on any type of column.
    3. **COUNTBLANK** returns the number of empty cells (blanks or empty strings) in a column.
    4. **COUNTROWS** returns the number of rows in a table.
    5. **DISTINCTCOUNT** returns the number of distinct values of a column, blank value included if present.
    6. **DISTINCTCOUNTNOBLANK** returns the number of distinct values of a column, no blank value included.



**EVALUATE**

the commonly used EVALUATE syntax is 
```sql
    [DEFINE { MEASURE <tableName>[<name>] = <expression> }]
EVALUATE <table>
    [ORDER BY {<expression> [{ASC | DESC}]} [, ...]]
```

*Define Measure*: part is useful to define local measures used in the query

*Order by* controls the sort order

```sql
Evaluate
Filter(
    table
    table[column] > 1
)
order by 
    table[column2]
    table[column3] ASC(DESC)
```


**Filter**

to apply multiple condition in filter function, use **AND** function

```sql
Filter(
    table,
    AND(
        table[column1] > 1,
        table[column2] < 20
    )
)
```

**ALL & ALLEXCEPT**

1. functions: all, allexcept, allcrossfiltered, allnoblank, allselected

2. ALL

    ignore the filtes automatically intruducted by a report

    useful when compute percentages or ratios 

    举例:当你想计算某个产品再所有产品中销售额所占的比例时，ignore the filter on [产品] column, then it can calculate the total sell out qty of all products

    ```sql
    All Sales Amount :=
    SUMX (
    ALL ( Sales ),
    Sales[Quantity] * Sales[Net Price]
    )
    ```

    对有上下级关系的两个column(屏幕大小， 产品名称），如果对下级column（产品名称）使用all计算 max value, 然后在matrix table中拉出每个上级（屏幕大小）所对应的max value, 那么它依然可以得出该上级分组（75寸屏幕大小）中的max values, 而不是所有屏幕尺寸中的max values. 
    因为all 的是下级column，但是上级的filter还在，会找出上级对应的每个下级单位，再计算max value， 相当于这个 all 在这里并没有作用？



    the parameter of ALL need to be wither a table name or a list of column names, cannot be a table expression

    if apply **ALL** to a column, it will return the distinct values in this columns.

    if apply **ALL** to a group of columns (must be in the same table), it will return the distinct combinaiton of those columns
2. ALLEXCEPT
    use when you want to include most, but not all the columns of a table.



**All Values & Distinct**

1. ALL always returns all the distinct values of a column.
2. VALUES returns only the distinct **visible** values. -- useful when using meausres, will react to the filter on tables
3. Distonct **not** consider BLANK() values
4. note:
engine automatically creates a lank row in any table that is on the one-side of a relationship in case the realtionship id invalid, but if the relationship is valid then the empty row will not be added.


**HASONEVALUE & SELECTEDVALUE**
1. hasonevalue: check whether the column has only one value selected 
    ```sql
    Brand Name :=
    IF (
    HASONEVALUE ( 'Product'[Brand] ),
    VALUES ( 'Product'[Brand] )
    )
    ```
    this one will check if there is one value selected, if so , it will return the brand name (value return the distict list), otherwise it will return blank()

2. selectedvalue: if there is only one value selected, it will return the value as scaler value, otherwise it is also possible return a defined defauly value
    ```sql 
    Brand Name := SELECTEDVALUE ( 'Product'[Brand], "Multiple brands" )
    ```
    this return the brand name when down to brand level, but at upper level(multiple brands in this group) it will return defined value "multiple brands"


#### Calcualted Tables and Measures

1. when using *Improt Model*, all calculated columns will occupy space in memory and are computed during table processing.
    
    pro: good when create calcualted columns, only one time calcualtion, will not spend time everytime when query it
    
    tip: try to avoid make intermediate calcualted columns because they will take extra RAM.


2. When using *Direct Query*, calculated columns are computed on the fly when the tabular engine queries the data source, this might rslult in heavy queries executed by the data source and producing **Slow Model**.

3. when you need to operate on aggregated values instead of operating on a row-by-row basisi, must use **Measures**

4. Calculated Columns are computed based on the row level context but measures is computed based on the context of visual element in DAX expression

5. Measures can be usd in Calculated columns and calcualted columns can be used in Measures



#### Variables

When creating Measure, try to avoid repeatly calling the same fields. 

Use Var and Return 